In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

from tqdm import tqdm

pd.set_option('display.max_columns', 30)

In [ ]:
## 데이터 불러온 후 num_date_time 열 삭제
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/XGBoost/data/test.csv')
test = test.drop("num_date_time", axis = 1)

## 변수들을 영문명으로 변경
cols = ["num", "date_time", "temp", "rain", "wind", "hum"]
test.columns = cols

## 시간 관련 변수들 생성
date = pd.to_datetime(test.date_time)
test['hour'] = date.dt.hour
test['day'] = date.dt.weekday
test['month'] = date.dt.month
test['week'] = date.dt.weekofyear
test['sin_time'] = np.sin(2*np.pi*test.hour/24)
test['cos_time'] = np.cos(2*np.pi*test.hour/24)
test['holiday'] = test.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)
test.loc[('2020-08-17'<=test.date_time)&(test.date_time<'2020-08-18'), 'holiday'] = 1

## 건물별, 요일별, 시간별 발전량 평균 / 건물별, 시간별 발전량 평균 및 표준편차 / 건물별 요일별 발전량 평균 및 표준편차 추가하기 위한 작업
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/XGBoost/data/train.csv')
train = train.drop("num_date_time", axis = 1)

cols = ["num", "date_time", "temp", "rain", "wind", "hum", "sun_hr", "sun_amount", "power"]
train.columns = cols

date = pd.to_datetime(train.date_time)
train['hour'] = date.dt.hour
train['day'] = date.dt.weekday
train['month'] = date.dt.month
train['week'] = date.dt.weekofyear

power_mean = pd.pivot_table(train, values = 'power', index = ['num', 'hour', 'day'], aggfunc = np.mean).reset_index()
power_hour_mean = pd.pivot_table(train, values = 'power', index = ['num', 'hour'], aggfunc = np.mean).reset_index()
power_hour_std = pd.pivot_table(train, values = 'power', index = ['num', 'hour'], aggfunc = np.std).reset_index()
power_day_mean = pd.pivot_table(train, values = 'power', index = ['num', 'day'], aggfunc = np.mean).reset_index()
power_day_std = pd.pivot_table(train, values = 'power', index = ['num', 'day'], aggfunc = np.std).reset_index()

## 건물별, 요일별, 시간별 발전량 평균
tqdm.pandas()
test['day_hour_mean'] = test.progress_apply(lambda x : power_mean.loc[(power_mean.num == x['num']) & (power_mean.day == x['day']) & (power_mean.hour == x['hour']) ,'power'].values[0], axis = 1)

## 건물별, 시간별 발전량 평균 및 표준편차
tqdm.pandas()
test['hour_mean'] = test.progress_apply(lambda x : power_hour_mean.loc[(power_hour_mean.num == x['num']) & (power_hour_mean.hour == x['hour']) ,'power'].values[0], axis = 1)

tqdm.pandas()
test['hour_std'] = test.progress_apply(lambda x : power_hour_std.loc[(power_hour_std.num == x['num']) & (power_hour_std.hour == x['hour']) ,'power'].values[0], axis = 1)

## 건물별, 요일별 발전량 평균 및 표준편차
tqdm.pandas()
test['day_mean'] = test.progress_apply(lambda x : power_day_mean.loc[(power_day_mean.num == x['num']) & (power_day_mean.day == x['day']), 'power'].values[0], axis = 1)

tqdm.pandas()
test['day_std'] = test.progress_apply(lambda x : power_day_std.loc[(power_day_std.num == x['num']) & (power_day_std.day == x['day']), 'power'].values[0], axis = 1)

## pandas 내 선형보간 method 사용
for i in range(100):
    test.iloc[i*168:(i+1)*168, :]  = test.iloc[i*168:(i+1)*168, :].interpolate()

## THI 변수 추가
test['THI'] = 9/5*test['temp'] - 0.55*(1-test['hum']/100)*(9/5*test['hum']-26)+32

# CDH 변수 추가
def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)

cdhs = np.array([])
for num in range(1,101,1):
    temp = test[test['num'] == num]
    cdh = CDH(temp['temp'].values)
    cdhs = np.concatenate([cdhs, cdh])
test['CDH'] = cdhs

## 체감온도 산출
test['sensory_temp'] = 13.12 + 0.6215*test['temp'] - 11.37*(test['wind']**0.16) + 0.3965*test['temp']*(test['wind']**0.16)

test.head()

<ipython-input-7-fb7db9e8d8cb>:14: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  test['week'] = date.dt.weekofyear
<ipython-input-7-fb7db9e8d8cb>:31: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  train['week'] = date.dt.weekofyear
100%|██████████| 16800/16800 [00:10<00:00, 1608.10it/s]


,num,date_time,temp,rain,wind,hum,hour,day,month,week,sin_time,cos_time,holiday,day_hour_mean,hour_mean,hour_std,day_mean,day_std,THI,CDH,sensory_temp
0,1,20220825 00,23.5,0.0,2.2,72,0,3,8,34,0.000000,1.000000,0,1627.80,1706.318118,446.882767,2704.598333,1158.47394,58.3456,-2.5,25.397063
1,1,20220825 01,23.0,0.0,0.9,72,1,3,8,34,0.258819,0.965926,0,1550.08,1622.620235,439.662704,2704.598333,1158.47394,57.4456,-5.5,25.201620
2,1,20220825 02,22.7,0.0,1.5,75,2,3,8,34,0.500000,0.866025,0,1431.12,1506.971294,412.071906,2704.598333,1158.47394,57.8725,-8.8,24.699788
3,1,20220825 03,22.1,0.0,1.3,78,3,3,8,34,0.707107,0.707107,0,1372.20,1437.365647,391.205981,2704.598333,1158.47394,57.9376,-12.7,24.136018
4,1,20220825 04,21.8,0.0,1.0,77,4,3,8,34,0.866025,0.500000,0,1381.72,1447.321412,381.099697,2704.598333,1158.47394,56.9961,-16.9,23.942400


In [ ]:
## 데이터 불러온 후 num_date_time 열 삭제
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/XGBoost/data/test.csv')
test = test.drop("num_date_time", axis = 1)

## 변수들을 영문명으로 변경
cols = ["num", "date_time", "temp", "rain", "wind", "hum", "sun_hr", "sun_amount"]
test.columns = cols

## 시간 관련 변수들 생성
date = pd.to_datetime(train.date_time)
train['hour'] = date.dt.hour
train['day'] = date.dt.weekday
train['month'] = date.dt.month
train['week'] = date.dt.weekofyear

#######################################
## 건물별, 요일별, 시간별 발전량 평균 넣어주기
#######################################
power_mean = pd.pivot_table(train, values = 'power', index = ['num', 'hour', 'day'], aggfunc = np.mean).reset_index()
tqdm.pandas()
train['day_hour_mean'] = train.progress_apply(lambda x : power_mean.loc[(power_mean.num == x['num']) & (power_mean.hour == x['hour']) & (power_mean.day == x['day']) ,'power'].values[0], axis = 1)

#######################################
## 건물별 시간별 발전량 평균 넣어주기
#######################################
power_hour_mean = pd.pivot_table(train, values = 'power', index = ['num', 'hour'], aggfunc = np.mean).reset_index()
tqdm.pandas()
train['hour_mean'] = train.progress_apply(lambda x : power_hour_mean.loc[(power_hour_mean.num == x['num']) & (power_hour_mean.hour == x['hour']) ,'power'].values[0], axis = 1)

#######################################
## 건물별 시간별 발전량 표준편차 넣어주기
#######################################
power_hour_std = pd.pivot_table(train, values = 'power', index = ['num', 'hour'], aggfunc = np.std).reset_index()
tqdm.pandas()
train['hour_std'] = train.progress_apply(lambda x : power_hour_std.loc[(power_hour_std.num == x['num']) & (power_hour_std.hour == x['hour']) ,'power'].values[0], axis = 1)

#######################################
## 건물별 요일별 발전량 평균 넣어주기
#######################################
power_day_mean = pd.pivot_table(train, values = 'power', index = ['num', 'day'], aggfunc = np.mean).reset_index()
tqdm.pandas()
train['day_mean'] = train.progress_apply(lambda x : power_day_mean.loc[(power_day_mean.num == x['num']) & (power_day_mean.day == x['day']) ,'power'].values[0], axis = 1)

#######################################
## 건물별 요일별 발전량 표준편차 넣어주기
#######################################
power_day_std = pd.pivot_table(train, values = 'power', index = ['num', 'day'], aggfunc = np.std).reset_index()
tqdm.pandas()
train['day_std'] = train.progress_apply(lambda x : power_day_std.loc[(power_day_std.num == x['num']) & (power_day_std.day == x['day']) ,'power'].values[0], axis = 1)

## 공휴일 변수 추가
train['holiday'] = train.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)
train.loc[('2020-08-17'<=train.date_time)&(train.date_time<'2020-08-18'), 'holiday'] = 1

## https://dacon.io/competitions/official/235680/codeshare/2366?page=1&dtype=recent
train['sin_time'] = np.sin(2*np.pi*train.hour/24)
train['cos_time'] = np.cos(2*np.pi*train.hour/24)

## https://dacon.io/competitions/official/235736/codeshare/2743?page=1&dtype=recent
train['THI'] = 9/5*train['temp'] - 0.55*(1-train['hum']/100)*(9/5*train['hum']-26)+32

## 체감온도 산출
train['sensory_temp'] = 13.12 + 0.6215*train['temp'] - 11.37*(train['wind']**0.16) + 0.3965*train['temp']*(train['wind']**0.16)

## CDH 변수 추가
def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)

cdhs = np.array([])
for num in range(1,101,1):
    temp = train[train['num'] == num]
    cdh = CDH(temp['temp'].values)
    cdhs = np.concatenate([cdhs, cdh])
train['CDH'] = cdhs

In [ ]:
test

,num,date_time,temp,rain,wind,hum,hour,day,month,week,sin_time,cos_time,holiday,day_hour_mean,hour_mean,hour_std,day_mean,day_std,THI,CDH,sensory_temp
0,1,20220825 00,23.5,0.0,2.2,72,0,3,8,34,0.000000,1.000000,0,1627.800000,1706.318118,446.882767,2704.598333,1158.473940,58.3456,-2.5,25.397063
1,1,20220825 01,23.0,0.0,0.9,72,1,3,8,34,0.258819,0.965926,0,1550.080000,1622.620235,439.662704,2704.598333,1158.473940,57.4456,-5.5,25.201620
2,1,20220825 02,22.7,0.0,1.5,75,2,3,8,34,0.500000,0.866025,0,1431.120000,1506.971294,412.071906,2704.598333,1158.473940,57.8725,-8.8,24.699788
3,1,20220825 03,22.1,0.0,1.3,78,3,3,8,34,0.707107,0.707107,0,1372.200000,1437.365647,391.205981,2704.598333,1158.473940,57.9376,-12.7,24.136018
4,1,20220825 04,21.8,0.0,1.0,77,4,3,8,34,0.866025,0.500000,0,1381.720000,1447.321412,381.099697,2704.598333,1158.473940,56.9961,-16.9,23.942400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,20220831 19,22.5,0.0,0.9,84,19,2,8,35,-0.965926,0.258819,0,964.873846,1010.462118,161.399578,789.907692,306.653068,61.4824,-34.5,24.695934
16796,100,20220831 20,20.7,0.0,0.4,95,20,2,8,35,-0.866025,0.500000,0,882.184615,928.125176,137.566008,789.907692,306.653068,65.2725,-34.4,23.253851
16797,100,20220831 21,20.2,0.0,0.4,98,21,2,8,35,-0.707107,0.707107,0,779.095385,830.032941,128.300189,789.907692,306.653068,66.7056,-35.3,22.771886
16798,100,20220831 22,20.1,0.0,1.1,97,22,2,8,35,-0.500000,0.866025,0,663.267692,723.100235,112.464079,789.907692,306.653068,65.7281,-36.8,22.159549


In [ ]:
test.to_csv('/content/drive/MyDrive/Colab Notebooks/XGBoost/data/test_preprocessed.csv', index=False)